# System File Execution Location Anomaly
Detects a Windows program executable started in a suspicious folder

## Rule Content
```
- title: System File Execution Location Anomaly
  id: e4a6b256-3e47-40fc-89d2-7a477edd6915
  status: experimental
  description: Detects a Windows program executable started in a suspicious folder
  references:
  - https://twitter.com/GelosSnake/status/934900723426439170
  author: Florian Roth, Patrick Bareiss
  date: 2017/11/27
  tags:
  - attack.defense_evasion
  - attack.t1036
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\svchost.exe'
      - '*\rundll32.exe'
      - '*\services.exe'
      - '*\powershell.exe'
      - '*\regsvr32.exe'
      - '*\spoolsv.exe'
      - '*\lsass.exe'
      - '*\smss.exe'
      - '*\csrss.exe'
      - '*\conhost.exe'
      - '*\wininit.exe'
      - '*\lsm.exe'
      - '*\winlogon.exe'
      - '*\explorer.exe'
      - '*\taskhost.exe'
    filter:
      Image:
      - C:\Windows\System32\\*
      - C:\Windows\SysWow64\\*
      - C:\Windows\explorer.exe
      - C:\Windows\winsxs\\*
      - \SystemRoot\System32\\*
    condition: selection and not filter
  falsepositives:
  - Exotic software
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\svchost.exe OR *\\rundll32.exe OR *\\services.exe OR *\\powershell.exe OR *\\regsvr32.exe OR *\\spoolsv.exe OR *\\lsass.exe OR *\\smss.exe OR *\\csrss.exe OR *\\conhost.exe OR *\\wininit.exe OR *\\lsm.exe OR *\\winlogon.exe OR *\\explorer.exe OR *\\taskhost.exe) AND (NOT (process_path.keyword:(C\:\\Windows\\System32\\* OR C\:\\Windows\\SysWow64\\* OR C\:\\Windows\\explorer.exe OR C\:\\Windows\\winsxs\\* OR \\SystemRoot\\System32\\*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()